In [1]:
import tensorflow as tf
import numpy as np
import random, json, string, pickle,nltk
import keras
import keras.layers
import keras.models
import keras.optimizers
from keras.layers.wrappers import TimeDistributed
import keras.layers.embeddings
import keras.preprocessing.text
import keras.preprocessing.sequence
import keras.callbacks
import sqlite3

Using TensorFlow backend.


In [5]:
conn = sqlite3.connect("yummly.db")
#recipes = conn.execute("SELECT ID, CleanIngredients FROM Recipe WHERE ImageAvailable=1 AND English=1;").fetchall()
titles = conn.execute("SELECT ID, Title FROM Recipe WHERE ImageAvailable=1 AND English=1;").fetchall()
conn.close()

In [2]:
[ids, labels, food2id, id2food, class_weights, class_weights2] = pickle.load( open( "ids_labels_nomissing_500.p", "rb" ) )

In [7]:
def prepIngredients(ingredients_string):
    return [x.strip() for x in ingredients_string.split(";")]

#recipes = [tuple([x[0], prepIngredients(x[1])])for x in recipes]

In [8]:
titles = [tuple([x[0], prepIngredients(x[1])])for x in titles]

In [13]:
#Get matching subset list of titles with id, labels
ids_test = set(ids)
titles = ([rec for rec in titles if rec[0] in ids_test])

[('a0e1f558-e1d3-43c6-b10b-152e3cc26e09', ['BBQ Chicken Spaghetti Squash']),
 ('893042ba-72e4-47a9-92af-6509281e3c92',
  ['Pear, Gorgonzola and Pecan Brussels Sprout Slaw']),
 ('89ace45e-ba7e-4ce2-b2f8-ba8e1daf168a', ['No Churn Vanilla Bean Ice Cream']),
 ('12985577-36a2-4f19-831d-5c9c3616cb76', ['Flat Belly Detox water']),
 ('269ebd13-b039-437c-8755-9e6f3435e0fa', ['Carrot Apple Ginger Juice']),
 ('a95af1d2-b82b-4021-b94e-98b2e212725f', ['DIY Ranch Seasoning']),
 ('e0871a4e-5820-4524-8d1e-2bacd7377963',
  ['Hemp Chocolate Smoothie with Raspberries & Pomegranates']),
 ('43975358-8c79-42de-889d-c05f1fd7f24e',
  ['Strawberry Creamsicle Cocktail And Mocktail']),
 ('25f9e384-fcef-4692-b93c-9fd6ebeb0f6c', ['Creamy Mushroom Risotto']),
 ('5c96ebf2-737c-4095-b5e9-a4a45816fcbe',
  ['Homemade Strawberry Balsamic Vinaigrette Dressing']),
 ('16009a8e-bbd7-41e8-b7ce-13542e74acfa',
  ['How to make Coconut Matcha Latte (3 minutes-2 ways)']),
 ('a198bae6-c42a-4aa1-af3d-534dd0605ffb',
  ['Unbelievable

In [ ]:
size_pantry = 500
all_ingredients = np.array([item for sublist in recipes for item in sublist[1]])
unique_ing = np.unique(all_ingredients, return_counts = True)

argsort_results = np.argsort(unique_ing[1])

sorted_ing = unique_ing[0][argsort_results]
sorted_vals = unique_ing[1][argsort_results]
sorted_ing = sorted_ing[::-1]

id2food = sorted_ing[0:size_pantry]
food2id = {food: idx[0] for idx, food in np.ndenumerate(id2food)}

In [ ]:
ids = [] #[x[0] for x in recipes]
recipe_ingredients = [x[1] for x in recipes]
labels = [] #np.zeros([len(recipes), size_pantry])

class_frequency = [0] * size_pantry


# Then we build our training data. We're not going to include
# recipes that have ingredients not in the pantry
for i in range(0, len(recipes)):
    num_misses = 0
    current_recipe = [0] * size_pantry
    
    for j in range(0, len(recipe_ingredients[i])):
        if food2id.get(recipe_ingredients[i][j]) != None:
            food_id = food2id.get(recipe_ingredients[i][j])
            
            current_recipe[food_id] = 1
            class_frequency[food_id] = class_frequency[food_id] + 1
        else:
            num_misses = num_misses + 1
    
    if num_misses == 0 and sum(current_recipe) > 0:
        labels.append(current_recipe)
        ids.append(recipes[i][0])
            

In [ ]:
len(recipe_ingredients)

In [2]:
#pickle.dump([ids, titles, labels, food2id, id2food, class_weights, class_weights2], open( "ids_labels_nomissing_500_titles.p", "wb" ) )
[ids, titles, labels, food2id, id2food, class_weights, class_weights2] = pickle.load( open( "ids_labels_nomissing_500_titles.p", "rb" ) )

In [6]:
len(ids), len(titles)

(519835, 519835)

In [ ]:
#Grab just title names
titles_name = []
for i in titles:
    titles_name.append(i[1])
    
titles = [item[0] for item in titles_name]

In [36]:
len(titles)

519835

In [27]:
len(titles)

519835

In [28]:
 pickle.dump( [ids, titles, labels, food2id, id2food, class_weights, class_weights2], open( "ids_labels_nomissing_500_titles.p", "wb" ) )

In [3]:
#lower case
titles_lower = [ lower_t.lower() for lower_t in titles ] 

In [4]:
titles_lower[1]

'pear, gorgonzola and pecan brussels sprout slaw'

In [5]:
vocabularySize = 1000 
# Split Title into words, and define a vocabulary with the most common words.
tokenizer = keras.preprocessing.text.Tokenizer(nb_words = vocabularySize, \
                                               filters = '!"#$%&()*+,-./:;<=>?@\\^_`{|}~\t\n')
#Fit on titleStrings 
tokenizer.fit_on_texts(titles_lower)

# Convert the title names into sequences of word ids using our vocabulary.
titleSequences = tokenizer.texts_to_sequences(titles_lower)

# Keep dictionaries that map ids -> words, and words -> ids.
word2id = tokenizer.word_index
id2word = {idx: word for (word, idx) in word2id.items()}
# Find the sentence with most words.
maxTitleLength = max([len(seq) for seq in titleSequences]) 

#pad sequences
data = keras.preprocessing.sequence.pad_sequences(titleSequences, 
           maxlen = (maxTitleLength + 1), padding = 'post', truncating = 'post')



In [6]:
id2word[0] = 'END'
word2id['END'] = 0

In [55]:
titles[1312]

'Yum'

In [53]:
print(data.shape)  
print(" ".join([id2word[idx] for idx in data[1312]]))

(519835, 25)
END END END END END END END END END END END END END END END END END END END END END END END END END


In [7]:
#build model...

#Define Shape of input batchSize
#output of Previous model X maxTitleLength
recipes_title = keras.layers.Input(batch_shape=(None,500), name = "input")


# Build a matrix of size vocabularySize x 300 where each row corresponds to a "word embedding" vector.
# This layer will convert replace each word-id with a word-vector of size 300.
embeddings = keras.layers.embeddings.Embedding(vocabularySize, 300, name = "embeddings")(recipes_title)

# Pass the word-vectors to the LSTM layer.
# We are setting the hidden-state size to 512.
hiddenStates = keras.layers.SimpleRNN(512, return_sequences = True, 
                                      input_shape=(maxTitleLength, 300), name = "rnn")(embeddings)

# Apply a linear (Dense) layer of size 512 x 256 to the outputs of the LSTM at each time step.
denseOutput = TimeDistributed(keras.layers.Dense(vocabularySize), name = "linear")(hiddenStates)
predictions = TimeDistributed(keras.layers.Activation("softmax"), name = "softmax")(denseOutput)  


#Model
model = keras.models.Model(input = recipes_title, output = predictions)

#For Recurrent Neural Networks, a type of
# optimization called RMSprop is preferred instead of the standard SGD udpates.
model.compile(loss='sparse_categorical_crossentropy', optimizer = keras.optimizers.RMSprop(lr = 0.001))

print(model.summary()) # Convenient function to see details about the network model.


sample_inputs = labels[0:10]
sample_outputs = model.predict(sample_inputs)
print('input size', sample_inputs.shape)
print('output size', sample_outputs.shape)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 500)           0                                            
____________________________________________________________________________________________________
embeddings (Embedding)           (None, 500, 300)      300000      input[0][0]                      
____________________________________________________________________________________________________
rnn (SimpleRNN)                  (None, 500, 512)      416256      embeddings[0][0]                 
____________________________________________________________________________________________________
linear (TimeDistributed)         (None, 500, 1000)     513000      rnn[0][0]                        
___________________________________________________________________________________________

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
sample_inputs

In [78]:
print('input size', sample_inputs.shape)
print('output size', sample_outputs.shape)

NameError: name 'sample_inputs' is not defined

In [10]:
#Train Model
inputData = labels  
outputData = data  # words 2, 3, 4, ... , (n)

outputLabels = np.expand_dims(outputData, -1)

checkpointer = keras.callbacks.ModelCheckpoint(filepath="my_weights_RNN.hdf5", save_weights_only = True, \
                                               save_best_only = True, monitor = 'loss')
model.fit(inputData, outputLabels, batch_size = 256, nb_epoch = 10, callbacks = [checkpointer])

model.save_weights('my_recipe_model.hdf5')

Exception: Error when checking model target: expected softmax to have shape (None, 500, 1) but got array with shape (519835, 25, 1)

array([243, 699,   2, 104, 221, 754, 368,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)